<a href="https://colab.research.google.com/github/ewertonmth/Desafio_Alura---GeminiConfirma/blob/main/GeminiConfirma2_0_public.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalação e configuração do Gemini 2.0 Flash no Google Colab
!pip install -q google-generativeai

import google.generativeai as genai

# Configuração com a sua chave da API
genai.configure(api_key="SUA_API_KEY")  # ← Substitua por sua chave!

# Inicializa o modelo Gemini 2.0 Flash
modelo = genai.GenerativeModel(model_name="gemini-2.0-flash")

# Teste de conexão com o modelo
try:
    resposta = modelo.generate_content("Diga 'ok' se você está funcionando corretamente.")
    print("✅ Conectado com sucesso ao Gemini 2.0 Flash!")
    print("Resposta de teste:", resposta.text)
except Exception as e:
    print("❌ Erro ao conectar à API do Gemini:", e)

✅ Conectado com sucesso ao Gemini 2.0 Flash!
Resposta de teste: ok



In [1]:
import requests

# Função para buscar no Google Custom Search (adicione suas credenciais)
GOOGLE_CSE_ID = "SUA_CSE_ID" # ← Substitua por sua chave!

def google_search(query, api_key=GOOGLE_API_KEY, cse_id=GOOGLE_CSE_ID, num_results=3):
    """
    Realiza uma busca no Google Custom Search e retorna os títulos e links dos principais resultados.
    """
    url = "https://www.googleapis.com/customsearch/v1"
    params = {
        "key": api_key,
        "cx": cse_id,
        "q": query,
        "num": num_results
    }
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        items = response.json().get("items", [])
        resultados = []
        for item in items:
            titulo = item.get("title")
            link = item.get("link")
            resultados.append(f"{titulo}: {link}")
        return "\n".join(resultados) if resultados else "Nenhum resultado encontrado."
    except Exception as e:
        return f"Erro na busca Google: {e}"

NameError: name 'GOOGLE_API_KEY' is not defined

In [ ]:
# Função de requisição ao Gemini com temperatura
def gemini_request(prompt, model=modelo, temperature=0.7):
        """
        Envia um prompt para o modelo Gemini e retorna a resposta.
        Permite configurar a temperatura para ajustar o nível de criatividade.
        """
        try:
            response = model.generate_content(
                prompt,
                generation_config={"temperature": temperature}
            )
            return response.text.strip()
        except Exception as e:
            return f"[Erro na requisição Gemini: {e}]"

In [ ]:
# Agente 1: Geração do resumo (temperatura 0.3)
def gerar_resumo(texto, model=modelo):
    prompt = f"""
    Sua tarefa é resumir o seguinte conteúdo de forma objetiva e clara,
    preservando os fatos principais e removendo opiniões ou exageros.

    Texto original:
    \"\"\"{texto}\"\"\"

    Resuma o conteúdo acima em no máximo 5 linhas:
    """
    return gemini_request(prompt, model=model, temperature=0.3)

In [ ]:
def verificar_fatos(resumo, model=modelo):
        # 1. Realiza a busca no Google usando a sua função externa
        resultados_busca = google_search(resumo) # Usa sua função google_search e GOOGLE_CSE_ID

        # 2. Monta o prompt INCLUINDO os resultados da busca
        prompt = f"""
        Você é um verificador de fatos. Analise o seguinte resumo de uma notícia e diga se ele apresenta informações
        verdadeiras, falsas ou duvidosas com base no conhecimento geral, nas fontes confiáveis listadas abaixo e em informações públicas.

        Resumo da notícia:
        \"\"\"{resumo}\"\"\"

        Resultados da busca no Google:
        {resultados_busca} # Os resultados da busca são adicionados aqui

        Diga se o conteúdo parece confiável, falso ou impreciso. Justifique sua análise de forma objetiva.
        """
        # 3. Envia o prompt COMPLETO (com os resultados da busca) para o Gemini
        return gemini_request(prompt, model=model, temperature=0.7)

In [ ]:
# Agente 3: Análise de linguagem manipulativa (temperatura 0.7)
def analisar_linguagem(texto, model=modelo):
    prompt = f"""
    Analise o seguinte texto e identifique se ele contém linguagem manipulativa, sensacionalista ou tendenciosa,
    como apelos emocionais, exageros ou tentativa de influenciar o leitor.

    Texto:
    \"\"\"{texto}\"\"\"

    Indique os principais pontos de manipulação encontrados e explique brevemente.
    """
    return gemini_request(prompt, model=model, temperature=0.7)

In [ ]:
# Agente 4: Geração do parecer final (temperatura 0.8)
def gerar_parecer_final(resumo, verificacao, linguagem, model=modelo):
    prompt = f"""
    Com base nas informações a seguir, faça um parecer final sobre a confiabilidade da notícia,
    usando linguagem clara e objetiva.

    Resumo da notícia:
    \"\"\"{resumo}\"\"\"

    Verificação factual:
    \"\"\"{verificacao}\"\"\"

    Análise de linguagem:
    \"\"\"{linguagem}\"\"\"

    Parecer final:
    """
    return gemini_request(prompt, model=model, temperature=0.8)

In [ ]:
# Função que integra o fluxo completo
def analisar_texto_completo(texto):
    resumo = gerar_resumo(texto)
    verificacao = verificar_fatos(resumo)
    linguagem = analisar_linguagem(texto)
    parecer = gerar_parecer_final(resumo, verificacao, linguagem)

    return {
        "Resumo": resumo,
        "Verificação Factual": verificacao,
        "Análise de Linguagem": linguagem,
        "Parecer Final": parecer
    }

In [ ]:
# Entrada do usuário
texto_teste = input("Digite o texto(ou link) da notícia ou conteúdo para análise:\n")

# Executa a análise
resultado = analisar_texto_completo(texto_teste)

# Exibe o resultado formatado
for etapa, texto in resultado.items():
    print(f"\n--- {etapa} ---\n{texto}")

Digite o texto da notícia ou conteúdo para análise:
https://www.diariodepernambuco.com.br/politica/2025/05/3893747-zema-posta-meme-de-lula-como-bebe-reborn-apertou-faz-caquinha.html

--- Resumo ---
O governador de Minas Gerais, Romeu Zema, publicou em suas redes sociais um meme que compara o presidente Lula a um bebê reborn, com a legenda "Apertou faz caquinha". A publicação gerou críticas e debates online sobre o teor da mensagem e a postura do governador. O meme foi interpretado por alguns como uma crítica à gestão do governo federal. A assessoria de Zema não se pronunciou sobre o caso.

--- Verificação Factual ---
Com base no resumo da notícia e na ausência de resultados da busca (devido ao erro na API do Google), a análise é a seguinte:

*   **Conteúdo:** O resumo descreve um evento específico: uma publicação de um meme pelo governador Romeu Zema em suas redes sociais, comparando o presidente Lula a um bebê reborn. Menciona a repercussão da publicação e a falta de pronunciamento da